**Master en Bioinformática y Biología Computacional, UAM**
## **Minería de texto**
# **Práctica de laboratorio 3: Redes neuronales para análisis de sentimientos**


---



# ***Ejercicio 1 - FFNs***

# Carga y preprocesado de datos - One-hot encoding

In [1]:
# Montamos el drive
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
# Fijamos semillas para generación de números aleatorios
import numpy as np
import tensorflow as tf

np.random.seed(1)
tf.random.set_seed(2)

In [3]:
import re
import pandas as pd

# Utilizamos el Stanford Sentiment Treebank (SST) como corpus de prueba para análisis de sentimientos de textos
# Cargamos y procesamos los datos para tratarlos como un problema de clasificación binaria de sentimientos
# Convertimos la escala de sentimientos de [1,5] a {0,1}
def load_sst_dataset(file_path, label_map={0:0, 1:0, 2:None, 3:1, 4:1}):
    data = []
    with open(file_path) as f:
        for i, line in enumerate(f):
            instance = {}
            instance['label'] = label_map[int(line[1])]
            if instance['label'] is None:
                continue

            # Filtramos caracteres y etiquetas de parseo
            text = re.sub(r'\s*(\(\d)|(\))\s*', '', line)
            instance['text'] = text[1:]
            data.append(instance)
    data = pd.DataFrame(data)
    return data

data_folder = 'drive/My Drive/Colab Notebooks/mintex2526-lab3/data/'
training_set = load_sst_dataset(data_folder + 'sst_training.txt') # conjunto de datos de entrenamiento
validation_set = load_sst_dataset(data_folder + 'sst_validation.txt') # conjunto de datos de validación
test_set = load_sst_dataset(data_folder + 'sst_test.txt') # conjunto de datos de test

print('Instancias de entrenamiento: {}'.format(len(training_set)))
print('Instancias de de validación: {}'.format(len(validation_set)))
print('Instancias de de test: {}'.format(len(test_set)))

FileNotFoundError: [Errno 2] No such file or directory: 'drive/My Drive/Colab Notebooks/mintex2526-lab3/data/sst_training.txt'

In [ ]:
from tensorflow.keras.preprocessing import text
from sklearn.utils import shuffle

# Desordenamos los conjuntos de datos
training_set = shuffle(training_set)
validation_set = shuffle(validation_set)
test_set = shuffle(test_set)

# Separamos textos y etiquetas de los conjuntos de datos
training_texts = training_set.text
training_labels = training_set.label
validation_texts = validation_set.text
validation_labels = validation_set.label
test_texts = test_set.text
test_labels = test_set.label

# Construimos un índice (vocabulario) para las 1000 palabras más frecuentes en el conjunto de datos de entrenamiento
tokenizer = text.Tokenizer(num_words=1000)
tokenizer.fit_on_texts(training_texts)

print("10 palabras del vocabulario con mayor frecuencia:")
words_index = tokenizer.word_index
for word, index in list(words_index.items())[:10]:
    count = tokenizer.word_counts[word]
    print(f"\t{word}  (índice: {index}, frecuencia: {count})")
print()

print("10 palabras del vocabulario con menor frecuencia:")
sorted_word_counts = sorted(tokenizer.word_counts.items(), key=lambda x: x[1], reverse=False)
last_words_counts = sorted_word_counts[:10]
for word, count in last_words_counts:
    index = tokenizer.word_index[word]
    print(f"\t{word}  (índice: {index}, frecuencia: {count})")

In [ ]:
# Vectorizamos los textos mediante la representación one-hot encoding
x_training = tokenizer.texts_to_matrix(training_texts, mode='binary')
y_training = training_labels
x_validation = tokenizer.texts_to_matrix(validation_texts, mode='binary')
y_validation = validation_labels
x_test = tokenizer.texts_to_matrix(test_texts, mode='binary')
y_test = test_labels

print('Dimensiones del conjunto de entrenamiento: {}'.format(x_training.shape))
print('Dimensiones del conjunto de validación: {}'.format(x_validation.shape))
print('Dimensiones del conjunto de test: {}'.format(x_test.shape))

In [ ]:
# Comprobación de algunos vectores generados:
# Iteramos sobre n textos (de entrenamiento) y sus vectores correspondientes
n = 3

index_to_word = tokenizer.index_word # Mapeos de índices a palabras en el vocabulario
word_counts = tokenizer.word_counts # Frecuencias de las palabras en los textos

for i, (text, text_vector) in enumerate(zip(training_texts[:n], x_training[:n]), start=1):
    print(f"Texto {i}: {text}")
    print(f"Vector: {text_vector}")
    print("Palabras:")
    for index, value in enumerate(text_vector):
        # Si el valor en el vector es 1, obtenemos la palabra correspondiente, junto con su índice y frecuencia en el vocabulario
        if value == 1:
            word = index_to_word.get(index)
            word_frequency = word_counts.get(word, 0) if word else 0
            print(f"\t{word} (índice={index}, frecuencia={word_frequency})")
    print()

# Construcción de la red

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

input_size = x_training[0].shape[0] # la longitud del vector de entrada es igual al tamaño del vocabulario

# Definimos una red neuronal feedforward con una única capa oculta de 16 unidades
# y ReLU como función de activación
model = Sequential()
model.add(Dense(units=16, activation='relu', input_shape=(input_size,))) # Nota: no es necesario indicar input_shape en capas sucesivas
model.add(Dense(units=1, activation='sigmoid'))

# Compilamos la red usando la entropía cruzada binaria como función de pérdida,
# el algoritmo Adam como optimizador del descenso por gradiente,
# y accuracy como métrica de evaluación en entrenamiento y validación
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# Opcionalmente se pueden utilizar otras funciones de activación (e.g., tanh, sigmoid) en vez de ReLU
# y otras funciones de pérdida como mean squared error (MSE)

In [ ]:
# Entrenamos la red durante 100 épocas y con batches de tamaño 32
history = model.fit(x_training, y_training, epochs=100, batch_size=32, validation_data=(x_validation, y_validation), verbose=1)

**Ejercicio 1a - Red de una capa oculta**

In [ ]:
import matplotlib.pyplot as plt

# Visualizamos un gráfica con el LOSS alcanzado en cada época para los conjuntos de entrenamiento y de validación
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('FNN 1 capa')
plt.ylabel('loss')
plt.xlabel('época')
plt.legend(['entrenamiento', 'validación'], loc='upper left')
plt.show()

# Visualizamos un gráfica con el ACCURACY alcanzado en cada época para los conjuntos de entrenamiento y de validación
# TO DO

**Ejercicio 1b - Red multicapa**

In [ ]:
# Añadimos una capa oculta adicional al modelo
model = Sequential()
# TO DO

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

history_2 = model.fit(x_training, y_training, epochs=100, batch_size=32, validation_data=(x_validation, y_validation), verbose=1)

# Visualizamos una gráfica con la evolución de LOSS alcanzado en cada época para los conjuntos de entrenamiento y de validación
# TO DO

In [ ]:
# Visualizamos una gráfica comparativa de la evolución de LOSS por épocas usando una o dos capas ocultas
plt.title('FNN 1 capa vs. 2 capas')
# TO DO


**Ejercicio 1c - Evaluación de hiperparámetros mediante grid search**

In [ ]:
# Comentario: para generar un modelo configurado con una serie de parámetros dada se aconseja implementar una función get_model, 
# que recibe tales parámetros como argumentos de entrada, y retorna el modelo correspondiente como salida

from tensorflow.keras.optimizers import Adam
import time
lrs = [0.0001, 0.001, 0.01] # valores de la learning rate
regs = [0.0, 0.0001] # valores de regularización L2
drs = [0.0, 0.2, 0.5] # valores de dropout

num_epochs = 100
early_stop = EarlyStopping(monitor='val_loss', patience=1) # para parar el entrenamiento antes de num_epochs atendiendo al error en validación

# Imprimimos por pantalla los valores valores de LOSS y ACCURACY de entrenamiento y validación de cada modelo, época a epóca
histories = {}
for lr in lrs:
    for reg in regs:
        for dr in drs:
            # TO DO

In [ ]:
# Obtenemos y visualizamos los parámetros de la mejor época atendiendo a los valores de ACCURACY en validación
# TO DO

# ***Ejercicio 2 - LSTMs***

# Preprocesado de datos - Tokenization y sequence padding

In [ ]:
from tensorflow.keras import preprocessing

# Creamos un tokenier que considera las 10000 palabras más frecuentes en el corpus
max_words = 10000
tokenizer = preprocessing.text.Tokenizer(num_words=max_words)

# Construimos el índice (diccionario) de palabras a partir del conjunto de datos de entrenamiento
tokenizer.fit_on_texts(training_texts)
word_index = tokenizer.word_index

# Obtenemos los textos como secuencias de enteros
trainining_sequences = tokenizer.texts_to_sequences(training_texts)
validation_sequences = tokenizer.texts_to_sequences(validation_texts)
test_sequences = tokenizer.texts_to_sequences(test_texts)

# Padding: convertimos las secuencias de enteros a un tensor 2D de forma (numero_secuencias, longitud_secuencia)
# Con ello igualamos la longitud de las secuencias de entrada
max_seq = 40
x_training = preprocessing.sequence.pad_sequences(trainining_sequences, maxlen=max_seq)
x_validation = preprocessing.sequence.pad_sequences(validation_sequences, maxlen=max_seq)
x_test = preprocessing.sequence.pad_sequences(test_sequences, maxlen=max_seq)

y_traininig = training_labels
y_validation = validation_labels
y_test = test_labels

print('Dimensiones del conjunto de entrenamiento: {}'.format(x_training.shape))
print('Dimensiones del conjunto de validación: {}'.format(x_validation.shape))
print('Dimensiones del conjunto de test: {}'.format(x_test.shape))

# Visualizamos un par de instancias de entrenamiento
print('\nTexto: {}\nVector padded: {}'.format(training_texts.iloc[0], x_training[0]))
print('\nTexto: {}\nVector padded: {}'.format(training_texts.iloc[1], x_training[1]))

# Construcción de la red

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM

embedding_size = 128
lstm_hidden_size = 128

model = Sequential()

# Añadimos una capa de embeddings
# Usamos mask_zero=True para ignorar los token '0' en el padding
# Tras la capa de embeddings, las activaciones tienen la forma (batch_size, max_seq, embedding_size)
model.add(Embedding(max_words, embedding_size, mask_zero=True))

# Añadimos una capa LSTM
model.add(LSTM(lstm_hidden_size))

# Añadimos una capa de salida
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
# Entrenamos la red durante 20 épocas y con batches de tamaño 128
history_lstm = model.fit(x_training, y_training, epochs=20, batch_size=128, validation_data=(x_validation, y_validation), verbose=1)

**Ejercicio 2a - LSTM de una capa**

In [ ]:
import matplotlib.pyplot as plt

# Visulizamos la evolución del LOSS y el ACCURACY de entrenamiento y validación por épocas de entrenamiento
# TO DO

# Evaluamos el modelo en test
score = model.evaluate(x_test, y_test, verbose=1)
print("Accuracy: ", score[1])

**Ejercicio 2b - Tamaño de embeddings y número de unidades LSTM**



Prueba diferentes tamaños de embeddings y número de unidades LSTM.
¿Observas alguna diferencia en las curvas de pérdida y precisión?
¿Qué sucede con un tamaño de embeddings muy pequeño (por ejemplo, 8) o unidades LSTM (por ejemplo, 16)? ¿Y qué ocurre cuando hacemos lo contrario?



In [ ]:
# TO DO

# ***Ejercicio 3 - LSTMs avanzadas***

**Ejercicio 3a - Stacked LSTMs**

In [ ]:
# Creamos un stack con capa de entrada de embeddings, dos capas intermedias (ocultas) de LSTMs, y una capa de salida con función de activación sigmoidal
model = Sequential()
# TO DO

# Usamos el optimizador RMSProp (Root Mean Square Propagation) appropiado para RNNs
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.summary()

# Entrenamos el modelo
history_lstm_stacked = model.fit(x_training, y_training, epochs=10, batch_size=128, validation_data=(x_validation, y_validation), verbose=1)

plt.plot(history_lstm_stacked.history['loss'])
plt.plot(history_lstm_stacked.history['val_loss'])
plt.title('Stacked LSTM')
plt.ylabel('loss')
plt.xlabel('época')
plt.legend(['entrenamiento', 'validación'], loc='upper left')
plt.show()

# Evaluamos el modelo en test
score = model.evaluate(x_test, y_test, verbose=1)
print("Accuracy: ", score[1])

**Ejercicio 3b - Bidirectional LSTMs**

In [ ]:
from tensorflow.keras.layers import Bidirectional

max_words = 10000
embedding_size = 50

# Creamos una red con una capa de entrada de embeddings, una capa intermedia (oculta) con una biLSTM de tamaño 128, y una capa de salida con función de activación sigmoidal
model = Sequential()
# TO DO

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# Entrenamos el modelo
history_bidirectional = model.fit(x_training, y_training, epochs=10, batch_size=128, validation_data=(x_validation, y_validation), verbose=2)

plt.plot(history_bidirectional.history['loss'])
plt.plot(history_bidirectional.history['val_loss'])
plt.title('LSTM bidirectional')
plt.ylabel('loss')
plt.xlabel('épocas')
plt.legend(['entrenamiento', 'validación'], loc='upper left')
plt.show()

# Evaluamos el modelo en test
score = model.evaluate(x_test, y_test, verbose=1)
print("Accuracy: ", score[1])